In [ ]:
#!jupyter nbconvert --to=python DO2_WOA_Comparison_python3.ipynb

In [ ]:
class DO2comp:
    '''
    class DO2comp
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 WOAvar='o_an',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit = 100,
                 cmap_extension='max',
                 verbose=True,
                 plotting=True,
                 output=False,
                 Taylor=True):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm
        from plot_Taylor_normalized import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)
        
        # load FESOM data -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DO2.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DO2 [mmol O2 m$^{-3}$]' 
        
        # load data -------------------------------------------------------------------------------------
        DO2fesom = pf.get_data(resultpath, "O2", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)


        # load WOA data  -------------------------------------------------------------------------------------
        WOA_input = WOAdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.WOAvar, get_overview=False)
        O2_int = WOA_input.woa_int     
        
        labelwoa = 'WOA'
        unitwoa = 'DO2 [mmol O2 m$^{-3}$]' #??? ## o_an:units = "micromoles_per_kilogram"

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        O2_int_ma = np.copy(O2_int)
        O2_int_ma[DO2fesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(self.verbose):
                    print('\nInput depth = {0}, plotting at level depth = {1} m\nFESOM min = {2:4.4f}, max = {3:4.4f}\nWOA min = {4:4.4f}, max = {5:4.4f}'.format(
                        d,plot_depth,
                        np.nanmin(DO2fesom[:,i]),np.nanmax(DO2fesom[:,i]),
                        np.nanmin(O2_int_ma[:,i]),np.nanmax(O2_int_ma[:,i])))
                    
                if plotting:
                    fig = plt.figure(figsize=(15,15), constrained_layout=True)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(0,341,10)
                    f1 = pf.subplot(mesh, m1, [DO2fesom[:,i]],
                                levels = levels,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelfesom+'\n ({0} m)'.format(depth_limit),
                                box=box,
                               )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, m2, [O2_int_ma[:,i]], 
                                levels = levels,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelwoa+'\n ({0} m)'.format(depth_limit),
                                box=box,
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.55, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-60,61,5)
                    f3 = pf.subplot(mesh, m3, [DO2fesom[:,i]-O2_int_ma[:,i]], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - WOA\n ({0} m)'.format(depth_limit),
                                box=box,
                               )

                    fig.subplots_adjust(bottom=0.02)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}m.png'.format(depth_limit), 
                                dpi = 300, bbox_inches='tight')
                    plt.show(block=False)
                
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------
                    # preparation of datasets
                    if np.isnan(np.min(O2_int_ma)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                    if np.isnan(np.min(DO2fesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Normalized Taylor Diagram for Temperature'
                    plt_Taylor_comp(O2_int_ma,DO2fesom,mask=True,title=title, depth_array=depth_array, mesh=mesh,verbose = self.verbose)
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DO2_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_layerwise.png', 
                                        dpi = 300, bbox_inches='tight')
                    plt.show(block=False)
                    
                if output:
                    print('Only return non-layerwise output')

        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - WOA_input.layer_depths[i_depth_limit]
            
            # print overview of chosen depth
            if(self.verbose):
                print('DO2 as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                    depth_limit_layer,depth_limit_level,i_depth_limit))
            
            # mean over depth
            DO2fesom_mean = np.mean(DO2fesom[:,:i_depth_limit], axis = 1)
            O2_int_ma_mean = np.mean(O2_int_ma[:,:i_depth_limit], axis = 1)
            
            if(self.verbose):
                print('\nFESOM min = {0:4.4f}, max = {1:4.4f}\nWOA min = {2:4.4f}, max = {3:4.4f}'.format(
                    np.nanmin(DO2fesom_mean),np.nanmax(DO2fesom_mean),
                    np.nanmin(O2_int_ma_mean),np.nanmax(O2_int_ma_mean)))
    
            if plotting:
                fig = plt.figure(figsize=(15,15), constrained_layout=True)
                axes = fig.subplot_mosaic(
                            """
                            AB
                            CC
                            """,
                            gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                            subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(0,341,10)
                f1 = pf.subplot(mesh, m1, [DO2fesom_mean],
                            levels = levels,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom+'\n (0-{0} m)'.format(depth_limit),
                            box=box,
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, m2, [O2_int_ma_mean], 
                            levels = levels,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelwoa+'\n (0-{0} m)'.format(depth_limit),
                            box=box,
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.55, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-60,61,5)
                f3 = pf.subplot(mesh, m3, [DO2fesom_mean-O2_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - WOA'+'\n (0-{0} m)'.format(depth_limit),
                            box=box,
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'DO2_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)
                
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(O2_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                if np.isnan(np.min(DO2fesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                title = 'Taylor Diagram for DO$_2$ \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
                plt_Taylor_norm(O2_int_ma_mean,DO2fesom_mean,mask=True,title=title,verbose = self.verbose)

                # Taylor fig export  
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'DO2_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)  
                
            if output:
                    self.do2_fesom = DO2fesom_mean
                    self.do2_phc   = O2_int_ma_mean
        

